# Homework 1

Choose 2 different time series which represent financial data* (e.g. you can take open stock prices, close stock prices, net income)**. You will analyze and prepare in-sample and out-of-sample forecasts for each of the time series:

Perform 2 stationarity tests to check for stationarity.
Analyze autocorrelation of the time series with Ljung-Box test and ACF, PACF.
If the series turns out to be non-stationary, use a proper technique to reduce the series to stationary.
For the stationary (or reduced to stationary) series, choose proper ARMA model and build an in-sample forecast and out-of-sample forecast for future 10 periods.
If you didn’t have to reduce your time series to stationary – skip this step. Otherwise, for initially non-stationary series, choose proper ARIMA model and build an in-sample forecast and out-of-sample forecast for future 10 periods.
Prepare at least 3 useful regressors (both dummy and continuous - in total, there should be 3), choose ARMAX or ARIMAX*** model and build an in-sample forecast and out-of-sample forecast for future 10 periods.
Choose the best model for your time series based on chosen (by you) quality metric.
For each of the chosen time series you can receive up to 5 points: 5 points for the first one and 5 points for the second one. Remember to provide a textual analysis of obtained results and justify all you decision (e.g. answer such questions as: Why have I chosen such ARIMA orders? Why do I think this regressors is useful for my time series? Why have I used differencing to reduce my time series to stationary? Why have I chosen this quality metric for my out-of-sample forecasts? How does my model behave for in-sample forecasts? etc.) In the very end, you should also provide your thoughts on why your chosen best model performed the best in case of this particular time series.

As a result of this homework you should provide a .ipynb file (jupyter notebook) with your code and comments. Also add files with your data (or you can add links and instructions, how to get your data, to your ipynb file).



*You can take data of any frequency – daily, monthly and so on.

**You can not take Microsoft stock prices or net income (as it has been discussed during our lessons).

***You will learn ARMAX and ARIMAX models on lessons 7-8.